In [1]:
import cv2
import face_recognition
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

C:\Users\aZam\.conda\envs\cv\lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
img = cv2.imread(r"C:\Users\aZam\OneDrive\Desktop\cv_project\c0286520-800px-wm.jpg")
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
encodings = face_recognition.face_encodings(rgb_img)
if len(encodings) > 0:
    img_encoding = encodings[0] 
else:
    img_encoding = None
    print("No face detected")

In [3]:
img2 = cv2.imread(r"C:\Users\aZam\OneDrive\Desktop\cv_project\download.jpg")

# تحويل BGR إلى RGB
rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# استخراج الـ encodings
encodings2 = face_recognition.face_encodings(rgb_img2)

# التأكد من وجود وجه
if len(encodings2) > 0:
    img_encoding2 = encodings2[0]
else:
    img_encoding2 = None
    print("No face detected in image 2")

In [4]:
if img_encoding is not None and img_encoding2 is not None:
    result = face_recognition.compare_faces([img_encoding], img_encoding2)
    print("Result:", result)
else:
    print("One or both images have no detected face")

Result: [np.False_]


In [5]:
result = face_recognition.compare_faces([img_encoding], img_encoding2)
print("Result:", result)

Result: [np.False_]


In [6]:
cv2.imshow("img", img)
cv2.imshow("img 2", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        images_path = glob.glob(os.path.join(images_path, "*.*"))
        print(f"{len(images_path)} encoding images found.")

        for img_path in images_path:
            img = cv2.imread(img_path)
            if img is None:
                continue

            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            filename = os.path.splitext(os.path.basename(img_path))[0]

            encodings = face_recognition.face_encodings(rgb_img)
            if len(encodings) == 0:
                print("No face in:", img_path)
                continue

            self.known_face_encodings.append(encodings[0])
            self.known_face_names.append(filename)

        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        if len(self.known_face_encodings) == 0:
            return np.empty((0, 4), dtype=int), []

        small_frame = cv2.resize(frame, (0, 0),
                                 fx=self.frame_resizing,
                                 fy=self.frame_resizing)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(
            rgb_small_frame, face_locations
        )

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(
                self.known_face_encodings, face_encoding
            )
            face_distances = face_recognition.face_distance(
                self.known_face_encodings, face_encoding
            )

            best_match_index = np.argmin(face_distances)
            name = "Unknown"

            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]

            face_names.append(name)

        face_locations = np.array(face_locations) / self.frame_resizing
        return face_locations.astype(int), face_names


In [9]:
import cv2

sfr = SimpleFacerec()
sfr.load_encoding_images(
    r"C:\Users\aZam\OneDrive\Desktop\cv_project"
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    face_locations, face_names = sfr.detect_known_faces(frame)

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


3 encoding images found.
Encoding images loaded
